# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get to the assignment --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [1]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [7]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "qwen2.5:14b"
MODEL1 = "gemma3:12b"
MODEL2 = "llama3:8b"

In [3]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [4]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [ ]:
# Let's just make sure the model is loaded

!ollama pull llama3.2

In [5]:
# If this doesn't work for any reason, try the 2 versions in the following cells
# And double check the instructions in the 'Recap on installation of Ollama' at the top of this lab
# And if none of that works - contact me!

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

Generative Artificial Intelligence (AI) technologies have broad applications across various industries due to their ability to create new content, designs, or ideas from existing data. Here are some key business applications:

1. **Content Creation and Marketing**: Generative AI can help businesses automate the creation of marketing materials such as product descriptions, social media posts, blog articles, and even images and videos. This not only saves time but also ensures that content is tailored to specific target audiences or markets.

2. **Product Design and Development**: In industries like fashion, automotive, and consumer goods, generative AI can assist in designing products by suggesting new designs based on trends, customer preferences, and market demand. It can also be used for rapid prototyping and customization of product offerings.

3. **Customer Service and Support**: Chatbots powered by generative AI can provide personalized support to customers across various channels

# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [8]:
import ollama

response = ollama.chat(model=MODEL1, messages=messages)
print(response['message']['content'])

Okay, let's break down the business applications of Generative AI. It's a *huge* and rapidly evolving area, so I'll structure this by category (Content Creation, Customer Service, Product Development, Operations & Automation, and Emerging/Niche Uses) and provide specific examples.  I'll also include a section at the end about considerations/caveats.

**1. Content Creation & Marketing**

*   **Copywriting & Ad Generation:**
    *   **Generating Ad Copy:**  Tools like Jasper, Copy.ai, and others can write ad headlines, body text, and call-to-actions for various platforms (Google Ads, Facebook, Instagram, etc.).  This significantly speeds up A/B testing and campaign creation.
    *   **Blog Post Creation:** Generative AI can create entire blog posts from a topic outline or brief.  (Requires careful editing and fact-checking – see caveats below).
    *   **Social Media Content:**  Creating captions, tweets, and other short-form content for social media platforms, often tailored to specific

## Alternative approach - using OpenAI python library to connect to Ollama

In [9]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL2,
    messages=messages
)

print(response.choices[0].message.content)

Generative AI has numerous business applications across various industries, including:

1. **Content Generation**: Generative models can create high-quality content, such as articles, social media posts,product descriptions, and even entire books. This can be used for marketing, advertising, and customer engagement.
2. **Customer Service**: Chatbots powered by generative AI can respond to customers' inquiries in a human-like manner, providing personalized support and resolving issues efficiently.
3. **Product Design and Development**: Generative algorithms can aid in conceptualizing new products, exploring design options, and automating the design process.
4. **Data Augmentation**: Generative models can generate synthetic data that simulates real-world scenarios, enhancing machine learning model training and improving performance.
5. **Recommendation Systems**: By generating personalized recommendations based on user preferences and behavior, businesses can increase sales, customer sat

## Are you confused about why that works?

It seems strange, right? We just used OpenAI code to call Ollama?? What's going on?!

Here's the scoop:

The python class `OpenAI` is simply code written by OpenAI engineers that makes calls over the internet to an endpoint.  

When you call `openai.chat.completions.create()`, this python code just makes a web request to the following url: "https://api.openai.com/v1/chat/completions"

Code like this is known as a "client library" - it's just wrapper code that runs on your machine to make web requests. The actual power of GPT is running on OpenAI's cloud behind this API, not on your computer!

OpenAI was so popular, that lots of other AI providers provided identical web endpoints, so you could use the same approach.

So Ollama has an endpoint running on your local box at http://localhost:11434/v1/chat/completions  
And in week 2 we'll discover that lots of other providers do this too, including Gemini and DeepSeek.

And then the team at OpenAI had a great idea: they can extend their client library so you can specify a different 'base url', and use their library to call any compatible API.

That's it!

So when you say: `ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')`  
Then this will make the same endpoint calls, but to Ollama instead of OpenAI.

## Also trying the amazing reasoning model DeepSeek

Here we use the version of DeepSeek-reasoner that's been distilled to 1.5B.  
This is actually a 1.5B variant of Qwen that has been fine-tuned using synethic data generated by Deepseek R1.

Other sizes of DeepSeek are [here](https://ollama.com/library/deepseek-r1) all the way up to the full 671B parameter version, which would use up 404GB of your drive and is far too large for most!

In [10]:
!ollama pull deepseek-coder-v2:16b-lite-instruct-q4_0

pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â ‡ pulling manifest 
pulling 5ff0abeeac1d:   0% â–•                  â–� 1.6 MB/8.9 GB                  pulling manifest 
pulling 5ff0abeeac1d:   0% â–•                  â–�  15 MB/8.9 GB                  pulling manifest 
pulling 5ff0abeeac1d:   0% â–•                  â–�  28 MB/8.9 GB                  pulling manifest 
pulling 5ff0abeeac1d:   0% â–•                  â–�  35 MB/8.9 GB                  pulling manifest 
pulling 5ff0abeeac1d:   1% â–•                  â–�  51 MB/8.9 GB                  pulling manifest 
pulling 5ff0abeeac1d:   1% â–•                  â–�  65 MB/8.9 GB                  pulling manifest 
pulling 5ff0abeeac1d:   1% â–•                  â–�  72 MB/8.9 GB                  pulling manifest 
pulling 5ff0abeeac1d:   1% â–•                  â–�  86 MB/8.9 GB                  pul

In [ ]:
# This may take a few minutes to run! You should then see a fascinating "thinking" trace inside <think> tags, followed by some decent definitions

response = ollama_via_openai.chat.completions.create(
    model="deepseek-r1:1.5b",
    messages=[{"role": "user", "content": "Please give definitions of some core concepts behind LLMs: a neural network, attention and the transformer"}]
)

print(response.choices[0].message.content)

# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI; use either of the above approaches.

In [ ]:
# Step 1: Create your prompts

system_prompt = "you are a personal assistant that write email replies in a professional manner but ultimately the answer should be no thank you."
user_prompt = """
Please draft a response to the following email: Hey Ben,

We recently launched a new platform at company A that can transform how your team work, and we’re seeing awesome  results for your target industry.

Do you handle data protection? If not, who is that person?

Thanks,

Jon Doe
"""

# Step 2: Make the messages list

messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]# fill this in

# Step 3: Call OpenAI

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

# Step 4: print the result

print(response)

In [ ]:
# Step 1: Create your prompts

system_prompt = "You are a genius assistant that explains things in a direct way"
user_prompt = """
What does the following python code mean and do? yield from set(book.get("author") for book in books if book.get("author"))

"""

# Step 2: Make the messages list

messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]# fill this in

# Step 3: Call OpenAI

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

# Step 4: print the result

print(response)